In [83]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import pandas as pd

In [84]:
paths = [
        'data/RBKvFKH20200920/2020-09-20 16_00_00 Rosenborg - Haugesund.xml',
         #'data/RBKvFKH20200920/haugesund_player-events_20-09-2020_rosenborg-haugesund.xml', 
         #'data/RBKvFKH20200920/haugesund_team-events_20-09-2020_rosenborg-haugesund.xml',
         #'data/RBKvFKH20200920/rosenborg_player-events_20-09-2020_rosenborg-haugesund.xml',
         #'data/RBKvFKH20200920/rosenborg_team-events_20-09-2020_rosenborg-haugesund.xml',
         #'data/RBKvFKH20200920/Rosenborg-Haugesund_20-09-2020_effective_time.xml'
         ]
         
dicts = []
for path in paths:
    tree = ET.parse(path)
    xml_data = tree.getroot()
    xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')
    dicts.append(dict(xmltodict.parse(xmlstr)))
    

In [85]:
paths

['data/RBKvFKH20200920/2020-09-20 16_00_00 Rosenborg - Haugesund.xml']

In [86]:
def sec_to_min(start, end):
    min_start = str(int(start)//60) + ',' + str(int(start)%60) + '\''
    min_end = str(int(end)//60) + ',' + str(int(end)%60) + '\''
    return min_start + '-' + min_end

In [103]:
rosenborg = []
for i in dicts[0]['file']['ROWS']['row']:
    if int(i['sort_order']) < 43 and 27 < int(i['sort_order']):
        rosenborg.append(i['code'][(i['code'].find(' ')+1):])
haugesund = []
for i in dicts[0]['file']['ROWS']['row']:
    if 42 < int(i['sort_order']):
        haugesund.append(i['code'][(i['code'].find(' ')+1):])
print(rosenborg)
print(haugesund)

['V. Hedenstad', 'A. Trondsen', 'G. Åsen', 'E. Tagseth', 'A. Hansen', 'K. Zachariassen', 'M. Lundemo', 'C. Holse', 'E. Konradsen Ceide', 'D. Islamović', 'P. Helland', 'E. Reitan', 'P. Skjelbred', 'E. Hovland', 'S. Adegbenro']
['K. Krygård', 'N. Sandberg', 'M. Desler', 'B. Hansen', 'C. Grindheim', 'I. Wadji', 'Bruno Leite', 'F. Knudsen', 'T. Pedersen', 'A. Stølås', 'H. Sandvik', 'K. Velde']


In [306]:
from collections import OrderedDict as odict
rbk = odict()
hfk = odict()
for name in rosenborg:
    rbk[name] = odict(({'total': 0, 'won':0, 'lost':0}))
for name in haugesund:
    hfk[name] = odict(({'total': 0, 'won':0, 'lost':0}))
duels = {} #dict with key = player number and value 3-list with [nr_duels, duels_won, duels_lost]
nr_duels = 0 
nr_won = 0
nr_lost = 0
for dictionary in dicts:
    for i in dictionary['file']['ALL_INSTANCES']['instance']:
        name = i['code'][(i['code'].find(' ')+1):]
        if name in rosenborg:
            if i['label'][-1]['group'] == 'duels' :
                for attr in i['label']:
                    if attr['text'] == 'Plus':
                        rbk[name]['total'] += 1
                        rbk[name]['won'] += 1
                    if attr['text'] == 'Minus':
                        rbk[name]['total'] += 1
                        rbk[name]['lost'] += 1
                    if attr['text'] == 'Neutral':
                        rbk[name]['total'] += 1
        elif name in haugesund:
            if i['label'][-1]['group'] == 'duels' :
                for attr in i['label']:
                    if attr['text'] == 'Plus':
                        hfk[name]['total'] += 1
                        hfk[name]['won'] += 1
                    if attr['text'] == 'Minus':
                        hfk[name]['total'] += 1
                        hfk[name]['lost'] += 1
                    if attr['text'] == 'Neutral':
                        hfk[name]['total'] += 1


In [312]:
for player in list(rbk.keys()):
    total = int(rbk[player]['total'])
    won = int(rbk[player]['won'])
    lost = int(rbk[player]['lost'])
    
    if won+lost > 0 and total > 0:
        rbk[player]['percentage'] = won/(won+lost)
        rbk[player]['percentage_of_total'] = total * rbk[player]['percentage']
        print(player, f'was in {total} duels and won {won}, lost {lost} or won {won/(won+lost):.2f}%')
    else:
        rbk[player]['percentage'] = 0
        rbk[player]['percentage_of_total'] = 0
for player in list(hfk.keys()):
    total = int(hfk[player]['total'])
    won = int(hfk[player]['won'])
    lost = int(hfk[player]['lost'])
    
    if won+lost > 0 and total > 0:
        hfk[player]['percentage'] = won/(won+lost)
        hfk[player]['percentage_of_total'] = total * hfk[player]['percentage']
        print(player, f'was in {total} duels and won {won}, lost {lost} or won {won/(won+lost):.2f}%')
    else:
        hfk[player]['percentage'] = 0
        hfk[player]['percentage_of_total'] = 0




V. Hedenstad was in 18 duels and won 8, lost 6 or won 0.57%
A. Trondsen was in 19 duels and won 8, lost 6 or won 0.57%
G. Åsen was in 17 duels and won 8, lost 7 or won 0.53%
E. Tagseth was in 17 duels and won 5, lost 7 or won 0.42%
K. Zachariassen was in 29 duels and won 12, lost 11 or won 0.52%
M. Lundemo was in 27 duels and won 9, lost 10 or won 0.47%
C. Holse was in 30 duels and won 8, lost 17 or won 0.32%
E. Konradsen Ceide was in 18 duels and won 6, lost 6 or won 0.50%
D. Islamović was in 32 duels and won 12, lost 11 or won 0.52%
P. Helland was in 7 duels and won 2, lost 3 or won 0.40%
E. Reitan was in 19 duels and won 11, lost 3 or won 0.79%
P. Skjelbred was in 13 duels and won 0, lost 7 or won 0.00%
E. Hovland was in 12 duels and won 2, lost 4 or won 0.33%
S. Adegbenro was in 22 duels and won 6, lost 12 or won 0.33%
K. Krygård was in 7 duels and won 2, lost 4 or won 0.33%
N. Sandberg was in 29 duels and won 5, lost 16 or won 0.24%
M. Desler was in 40 duels and won 14, lost 11 or

In [315]:
nested_rbk = []
nested_hfk = []
for player in list(rbk.keys()):
    nested_rbk.append(list(rbk[player].values()))
for player in list(hfk.keys()):
    nested_hfk.append(list(hfk[player].values()))
df_rbk = pd.DataFrame(nested_rbk, columns=rbk['V. Hedenstad'].keys(), index = rosenborg)
df_hfk =  pd.DataFrame(nested_hfk, columns=hfk['K. Velde'].keys(), index = haugesund)
    


In [316]:
import plotly.graph_objects as go
players = rosenborg
data = []
won = list(df_rbk['percentage_of_total'])
lost= list(df_rbk['total'])
for i in range(len(won)):
    lost[i] -= won[i]

fig = go.Figure(data=[
    go.Bar(name='won', x=players, y=won, marker=dict(color="white")),
    go.Bar(name='lost', x=players, y=lost, marker=dict(color="blue"))
])
# Change the bar mode
fig.update_layout(barmode='stack')

fig.show()





In [317]:
players = haugesund
won = list(df_hfk['percentage_of_total'])
lost= list(df_hfk['total'])
for i in range(len(won)):
    lost[i] -= won[i]

fig = go.Figure(data=[
    go.Bar(name='won', x=players, y=won, marker=dict(color="blue")),
    go.Bar(name='lost', x=players, y=lost, marker=dict(color="white")),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

